In [1]:
import heapq
from collections import deque
import time

class ColaPrioridadLimitada(object):
    def __init__(self, limite=None, *args):
        self.limite = limite
        self.queue = list()
    #DEVUELVE EL VALOR D ELA COLA
    def __getitem__(self, val):
        return self.queue[val]
    #cuanto es el tamañonde la cola
    def __len__(self):
        return len(self.queue)
    #Agrega elementos x es el valor saca el d emenos valor y agrega el nuevo
    def push(self, x):
        heapq.heappush(self.queue, x)
        if self.limite and len(self.queue) > self.limite:
            self.queue.remove(heapq.nlargest(1, self.queue)[0])
    # de que cola quieres sacar elementos 
    def pop(self):
        return heapq.heappop(self.queue)
    # para incluir
    def extend(self, iterable):
        for x in iterable:
            self.append(x)

    def clear(self):
        for x in self:
            self.queue.remove(x)

    def remove(self, x):
        self.queue.remove(x)

    # Metodo para saber si la cola esta vacia
    def empty(self):
        if not self.heap:
            return True
        else:
            return False
        
    def sorted(self):
        return heapq.nsmallest(len(self.queue), self.queue)
class NodoBusqueda(object):
    
    '''Nodo para el proceso de busqueda.'''

    def __init__(self, estado, padre=None, accion=None, costo=0, problema=None, profundidad=0):
        self.estado = estado
        self.padre = padre
        self.accion = accion
        self.costo = costo
        self.problema = problema or padre.problema
        self.profundidad = profundidad

    def expandir(self, busqueda_local=False):
        '''Crear sucesores.'''
        nodos_nuevos = []
        for accion in self.problema.acciones(self.estado):
            estado_nuevo = self.problema.resultado(self.estado, accion)
            costo = self.problema.costo(self.estado, accion, estado_nuevo)
            fabrica_nodos = self.__class__
            nodos_nuevos.append(fabrica_nodos(estado=estado_nuevo,
                                         padre=None if busqueda_local else self,
                                         problema=self.problema,
                                         accion=accion,
                                         costo=self.costo + costo,
                                         profundidad=self.profundidad + 1))
        return nodos_nuevos

    def camino(self):
        '''Camino (lista de nodos y acciones) desde el nodo raiz al nodo actual.'''
        nodo = self
        camino = []
        while nodo:
            camino.append((nodo.accion, nodo.estado))
            nodo = nodo.padre
        return list(reversed(camino))

    def __eq__(self, otro):
        return isinstance(otro, NodoBusqueda) and self.estado == otro.estado

    def estado_representacion(self):
        return self.problema.estado_representacion(self.estado)

    def accion_representacion(self):
        return self.problema.accion_representacion(self.accion)

    def __repr__(self):
        return 'Node <%s>' % self.estado_representacion().replace('\n', ' ')

    def __hash__(self):
        return hash((
            self.estado,
            self.padre,
            self.accion,
            self.costo,
            self.profundidad,
        ))
class NodoBusquedaHeuristicaOrdenado(NodoBusqueda):
    def __init__(self, *args, **kwargs):
        super(NodoBusquedaHeuristicaOrdenado, self).__init__(*args, **kwargs)
        self.heuristica = self.problema.heuristica(self.estado)

    def __lt__(self, otro):
        return self.heuristica < otro.heuristica
class NodoBusquedaAEstrellaOrdenado(NodoBusquedaHeuristicaOrdenado):
    def __lt__(self, otro):
        return self.heuristica + self.costo < otro.heuristica + otro.costo
class ProblemaBusqueda(object):
    '''Clase base abstracta, para representar y manipular los espacio de busqueda
    de un problema. IEn esta clase, el espacio de búsqueda debe representarse 
    implícitamente como un gráfico.
    Cada estado corresponde con un estado del problema(es decir, una configuración válida) 
    y cada accion del problema(es decir, una transformación válida a una configuración) corresponde con un limite o frontera.
    Para utilizar esta clase se debe implementar metodos requeridos by el algoritmo de busqueda
    que se utilizara.'''

    def __init__(self, estado_inicial=None):
        self.estado_inicial = estado_inicial

    def acciones(self, estado):
        '''Devuelve las acciones disponibles para realizar a partir de un estado.
        El valor devuelto es íterador sobre acciones.
        Las acciones son específicas del problema y no se debe asumir nada sobre ellas.
        '''
        raise NotImplementedError

    def resultado(self, estado, accion):
        '''Debuelve un nuevo estado despues de aplicar una accion a estado.'''
        raise NotImplementedError

    def costo(self, estado, accion, estado2):
        '''Devuelve el costo de aplicar una accion para alcanzar el estado2 a partir de estado.
            El valor devuelto es un numero (intero o de punto flotante),
            por defecto la funcion devuelve 1.
        '''
        return 1

    def es_objetivo(self, estado):
        '''Devuelve True si estado es el estado_objetivo y false caso contrario'''
        raise NotImplementedError

    def valor(self, estado):
        '''Devuelve el valor de `estado`, para motivos de optimizacion.
           valor es un numero (entero o punto flotante).'''
        raise NotImplementedError

    def heuristica(self, estado):
        '''DEvuelve un estimado del costo faltante para alcanzar la solucion a partir de `estado`.'''
        return 0

    def estado_representacion(self, estado):
        """
        Devuelve un string de representacion de un estado.
        Por defecto devuelve str(estado).
        """
        return str(estado)

    def accion_representacion(self, accion):
        """
        Devuelve un string de representacion de una acción.
        Por defecto devuelve str(acción).
        """
        return str(accion)
def aestrella(problema, busqueda_en_grafo=False, viewer=None):
    '''
    Busuqeda A*.

    Si se establece busqueda_en_grafo=True, se eliminara la busqueda en estados repetidos.
    Requiere redefinir las funciones de la clase ProblemaBusqueda:
    ProblemaBusqueda.acciones, ProblemaBusqueda.resultado, y
    ProblemaBusqueda.es_objetivo, ProblemaBusqueda.costo,
    ProblemaBusqueda.heuristica.
    '''
    return _buscar(problema,
                   ColaPrioridadLimitada(),
                   busqueda_en_grafo=busqueda_en_grafo,
                   fabrica_nodos=NodoBusquedaAEstrellaOrdenado,
                   reemplazar_grafo_cuando_mejor=True)

def _buscar(problema, frontera, busqueda_en_grafo=False, limite_profundidad=None,
            fabrica_nodos=NodoBusqueda, reemplazar_grafo_cuando_mejor=False):
    # le pasamos todod lo que implementamos
    '''
    Algoritmo basico de busqueda, base de todos los demas algoritmos de busqueda.
    '''
#mmemoria es igual a nodo visitado
    memoria = set()
#crea el nodo inicial de manera generalizada y lo pone a la frontera 
    nodo_inicio = fabrica_nodos(estado=problema.estado_inicial, problema=problema)
    frontera.push(nodo_inicio)
#mientras la frontera tenga valor 
    while frontera:
        #Saca el nodo de la frontera
        nodo = frontera.pop()
        #si el problema es objetivo(nodo estado)
        if problema.es_objetivo(nodo.estado):
            return nodo
        #obtenemos el nodo como resultado 
        #si no es el caso agregamos a memoria visitado el nodo con su estado 
        memoria.add(nodo.estado)
#expandimos los hijos (opcional )
        if limite_profundidad is None or nodo.profundidad < limite_profundidad:
            expandido = nodo.expandir()
#por cada nodod que expandido     
            for n in expandido:
            #preguntamos si nodo esta en busqueda en grafo 
                if busqueda_en_grafo:
                    #no los va sacar sueltos los nodos si x estado es igual an estado
                    otros = [x for x in frontera if x.estado == n.estado]
                    #garantizar que exista un resultado saca el tamaño de otros (opcional )
                    assert len(otros) in (0, 1)
                    #si n estado (revisison de ndosos que aunnno estan en la memomria o visitados )
                    #si no esta en la memrio y el tamaño d elos otros es igual 0
                    if n.estado not in memoria and len(otros) == 0:
                        #sigue en
                        frontera.push(n)
                    elif reemplazar_grafo_cuando_mejor and len(otros) > 0 and n < otros[0]:
                        frontera.remove(otros[0])
                        frontera.push(n)
                else:
                    frontera.push(n)
class EigthPuzzleProblem(ProblemaBusqueda):
    def acciones(self, estado):
        '''Devuelve una lista de piesas que se pueden mover a un espacio vacio.'''
        filas = string_to_list(estado)
        fila_e, columna_e = find_location(filas, 'o')

        acciones = []
        if fila_e > 0:
            acciones.append(filas[fila_e - 1][columna_e-1]) #arribaiz
           
        if columna_e < 3:
            acciones.append(filas[fila_e -1][columna_e + 1]) #arriba der
           
        return acciones

    def resultado(self, estado, accion):
        '''Devuelve el resultado despues de mover una pieza a un espacio en vacio
        '''
        filas = string_to_list(estado)
        fila_e, columna_e = find_location(filas, 'o')
        fila_n, columna_n = find_location(filas, accion)

        filas[fila_e][columna_e], filas[fila_n][columna_n] = filas[fila_n][columna_n], filas[fila_e][columna_e]

        return list_to_string(filas)

    def es_objetivo(self, estado):
        '''Devuelve True si un estado es el estado_objetivo.'''
        return estado == OBJETIVO

    def costo(self, estado1, accion, estado2):
        '''Devuelve el costo de ejecutar una accion. 
        '''
        return 1

    def heuristica(self, estado):
#         Devuelve una estimacion de la distancia de un estado a otro, utilizando la distancia manhattan.
        filas = string_to_list(estado)

        distancia = 0

        for numero in 'abcdefghijklmnño':
            fila_n, columna_n = find_location(filas, numero)
            fila_n_objetivo, col_n_goal = posiciones_objetivo[numero]

            distancia += abs(fila_n - fila_n_objetivo) + abs(columna_n - col_n_goal)

        return distancia
OBJETIVO = '''a-b-c-d
e-f-g-h
i-j-k-l
m-n-ñ-o'''

INICIAL ='''a-b-o-d
e-h-g-f
c-j-n-l
m-i-ñ-k'''

#INICIAL = '''1-2-3
#4-e-6
#7-8-5'''

def list_to_string(list_):
    return '\n'.join(['-'.join(row) for row in list_])

def string_to_list(string_):
    return [row.split('-') for row in string_.split('\n')]

def find_location(filas, element_to_find):
    '''Encuentra la ubicacion de una pieza en el rompecabezas.
       DEvuelve una tupla: fila, columna'''
    for ir, row in enumerate(filas):
        for ic, element in enumerate(row):
            if element == element_to_find:
                return ir, ic

posiciones_objetivo = {}
filas_objetivo = string_to_list(OBJETIVO)
for numero in 'abcdefghijklmnño':
    posiciones_objetivo[numero] = find_location(filas_objetivo, numero)

def main():
    
    problema = EigthPuzzleProblem(INICIAL)
    resultado = aestrella(problema)
   
    for accion, estado in resultado.camino():
     print('Move numero', accion)
     print(estado)
 


if __name__ == "__main__":
  start = time.time()
  main()
  end = time.time()
  
  print("Tiempo : ",end-start, "seg")

Move numero None
a-b-o-d
e-h-g-f
c-j-n-l
m-i-ñ-k
Move numero k
a-b-k-d
e-h-g-f
c-j-n-l
m-i-ñ-o
Move numero n
a-b-k-d
e-h-g-f
c-j-o-l
m-i-ñ-n
Move numero f
a-b-k-d
e-h-g-o
c-j-f-l
m-i-ñ-n
Move numero k
a-b-o-d
e-h-g-k
c-j-f-l
m-i-ñ-n
Move numero n
a-b-n-d
e-h-g-k
c-j-f-l
m-i-ñ-o
Move numero f
a-b-n-d
e-h-g-k
c-j-o-l
m-i-ñ-f
Move numero h
a-b-n-d
e-o-g-k
c-j-h-l
m-i-ñ-f
Move numero a
o-b-n-d
e-a-g-k
c-j-h-l
m-i-ñ-f
Move numero i
i-b-n-d
e-a-g-k
c-j-h-l
m-o-ñ-f
Move numero c
i-b-n-d
e-a-g-k
o-j-h-l
m-c-ñ-f
Move numero a
i-b-n-d
e-o-g-k
a-j-h-l
m-c-ñ-f
Move numero n
i-b-o-d
e-n-g-k
a-j-h-l
m-c-ñ-f
Move numero f
i-b-f-d
e-n-g-k
a-j-h-l
m-c-ñ-o
Move numero h
i-b-f-d
e-n-g-k
a-j-o-l
m-c-ñ-h
Move numero n
i-b-f-d
e-o-g-k
a-j-n-l
m-c-ñ-h
Move numero i
o-b-f-d
e-i-g-k
a-j-n-l
m-c-ñ-h
Move numero c
c-b-f-d
e-i-g-k
a-j-n-l
m-o-ñ-h
Move numero a
c-b-f-d
e-i-g-k
o-j-n-l
m-a-ñ-h
Move numero i
c-b-f-d
e-o-g-k
i-j-n-l
m-a-ñ-h
Move numero c
o-b-f-d
e-c-g-k
i-j-n-l
m-a-ñ-h
Move numero a
a-b-f-d
e-c-g-k
i